In [1]:
%run ..\notebooks\Util_func.ipynb

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
# SANDAG
REG = 'SANDAG'
base_dir = r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2018\SANDAG\2_tour_extract\wt_wkday'
raw_dir = r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2018\SANDAG'

out_file = r'out\%s_12_TNCTrips_IncVeh.xlsx' %REG

In [3]:
## Process household records
hh_df = pd.read_csv(join(raw_dir, 'ex_hh_wZones.csv'))
hh_df = hh_df[['hh_id','home_county_fips','num_people','num_vehicles',
              'num_hh_members_age_18_to_34','num_hh_members_age_35_to_64','num_hh_members_age_65_to_74',
              'num_hh_members_age_75_plus','num_workers']]
hh_df['num_adults'] = hh_df['num_hh_members_age_18_to_34'] + hh_df['num_hh_members_age_35_to_64'] +\
                        hh_df['num_hh_members_age_65_to_74'] + hh_df['num_hh_members_age_75_plus']
hh_df['veh_adult_ratio'] = hh_df['num_vehicles']/hh_df['num_adults']

hh_df['veh_adult_ratio_cat'] = 0
hh_df.loc[hh_df['veh_adult_ratio']>10, 'veh_adult_ratio_cat'] = 4
hh_df.loc[(hh_df['veh_adult_ratio']<1) & (hh_df['veh_adult_ratio_cat']==0), 'veh_adult_ratio_cat'] = 1
hh_df.loc[(hh_df['veh_adult_ratio']==1) & (hh_df['veh_adult_ratio_cat']==0), 'veh_adult_ratio_cat'] = 2
hh_df.loc[(hh_df['veh_adult_ratio']>1) & (hh_df['veh_adult_ratio_cat']==0), 'veh_adult_ratio_cat'] = 3
hh_df = hh_df[['hh_id','veh_adult_ratio_cat']]

## Process person records
tmp_df = pd.read_csv(join(raw_dir, 'ex_person_wZones.csv'))
tmp_df = tmp_df.merge(hh_df, how='left')
if REG=='SANDAG' or REG=='SCAG':
    tmp_df2 = pd.read_csv(join(raw_dir, 'ex2_person.tsv'), sep='\t')
    tmp_df = tmp_df.merge(tmp_df2[['hh_id','person_id','raceeth_new_imputed']], how='left')
tmp_df = tmp_df[['hh_id','person_num','raceeth_new_imputed','income_imputed','gender','age','veh_adult_ratio_cat']]
tmp_df = tmp_df.rename(columns={'raceeth_new_imputed':'raceeth','income_imputed':'hinc'})
tmp_df = tmp_df.rename(columns={'hh_id':'hhno','person_num':'pno'})

per_df = pd.read_csv(join(base_dir,'survey2018_precx.dat'), sep=' ')
per_df = per_df.merge(tmp_df, how='left')
per_df['count'] = 1
per_df = per_df[per_df['psexpfac']>0]

## Process trip records
raw_trips = pd.read_csv(join(raw_dir, 'ex_trip_wZones.csv'))
raw_trips = raw_trips[['hh_id','person_num','trip_num','mode_uber','mode_lyft','tnc_pooled','mode_type_imputed']]
raw_trips = raw_trips.rename(columns={'hh_id':'hhno','person_num':'pno','trip_num':'tsvid'})

trip_df = pd.read_csv(join(base_dir,'survey2018_tripx.dat'), sep=' ')

req_percols = ['hhno','pno','veh_adult_ratio_cat','hinc']

def prep_df(df):
    df = link_dt(df)
    
    df = df.loc[df['mode']==9, ]
    df = df.merge(raw_trips, how='left')
    df = df[df['mode_type_imputed']!=4] #remove taxi trips
    
    df['tnc_type'] = 3 # prem/other
    df.loc[(df['mode_uber']==1) | (df['mode_lyft']==1), 'tnc_type'] = 1 # pooled
    df.loc[(df['mode_uber']==2) | (df['mode_lyft']==2), 'tnc_type'] = 2 # regular
    
    df = df.merge(per_df[req_percols], 
                        how='left', on=['hhno','pno'])
    df['count'] = 1
    df = df[(df['trexpfac']>0) & (df['mode']>0)]
    df = df[(df['otaz']>0) & (df['dtaz']>0)]
    return df
    
trip_df = prep_df(trip_df)
reg = trip_df.loc[trip_df['tnc_type']==2, ]
pool = trip_df.loc[trip_df['tnc_type']==1, ]
prem = trip_df.loc[trip_df['tnc_type']==3, ]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [4]:
col_dict = {
    'hinc': {
        'desc': 'HHInc',
        'col': 'hinc',
        'vals': range(1,9),
        'labels': ['1_25K','2_25_50K','3_50_75K','4_75_100K','5_100_150K','6_150_200K','7_200_250K','8_250K']
    },
    'tncmode': {
        'desc': 'TNCMode',
        'col': 'tnc_type',
        'vals': range(1,4),
        'labels': ['2_Pool','1_Reg','3_PremOth']
    },
    'veh_adult_ratio_cat': {
        'desc': 'Veh_Adult_Ratio',
        'col': 'veh_adult_ratio_cat',
        'vals': range(1,5),
        'labels': ['1_Veh_LT_Adults','2_Veh_EQ_Adults','3_Veh_GT_Adults','4_Missing']
    }
}

In [5]:
fname = out_file
writer = pd.ExcelWriter(fname, engine='xlsxwriter')
workbook = writer.book
format1 = workbook.add_format({'num_format': '#,##0.0'})

In [6]:
from xlsxwriter.utility import xl_rowcol_to_cell

wt_cols = ['count', 'trexpfac']
wt_desc = ['(Unweighted)', '(Weighted)']

d1_dict = col_dict['veh_adult_ratio_cat']
d2_dict = col_dict['hinc']

for df, sname in zip([trip_df, reg, pool, prem], ['All_TNC', 'TNC_Reg', 'TNC_Pool', 'TNC_Prem']):
    row = 0
    for wc, wd in zip(wt_cols, wt_desc):
        title = 'TNC Trips by ' + d2_dict['desc'] + ' and ' + d1_dict['desc'] + ' ' + wd
        tab, tab_fmt = prep_data_2d(df,d1_dict['col'],d1_dict['vals'],d1_dict['labels'],
                                d2_dict['col'],d2_dict['vals'],d2_dict['labels'],wc)
        row = write_to_excel(tab.astype('float64'), sname, title, row)
        
        title = 'Column Shares by ' + d2_dict['desc'] + ' and ' + d1_dict['desc'] + ' ' + wd
        row = write_to_excel(getSharesIdx(tab.copy()), sname, title, row)
    tab_range = xl_rowcol_to_cell(row,1) + ':' + xl_rowcol_to_cell(row,tab.shape[1])
    _ = writer.sheets[sname].set_column(tab_range, 11, format1)

In [7]:
writer.save()